In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import h5py
import pickle
from pathlib import Path

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from src.data.file_utils import GetTV, _find_index

ImportError: cannot import name '_find_index' from 'src.data.file_utils' (/Users/nc1514/Documents/GitHub/plasma-tv/src/data/file_utils.py)

## Manual Point Labeling

In [ ]:
from PIL import Image, ImageTk
from tkinter import Tk, Label

from src.external.labeling import manual_labeling as ml

In [ ]:
file_idx = 2  # Index of the file to be loaded
root = Tk()
tv_image_path = Path("tv_images/l-mode")
output_path = Path("../outputs/manual_labeled_points")

tv = GetTV(tv_image_path)
files = tv.list_files()
file = files[file_idx]
inverted = tv.load(file, "inverted")
out_file = output_path / Path(file.stem).with_suffix(".csv")
photos = (255 * (inverted - np.min(inverted)) / (np.max(inverted) - np.min(inverted))).astype("uint8")

In [ ]:
current_photo_index = 0
photo = ImageTk.PhotoImage(image=Image.fromarray(photos[current_photo_index]))
photo = photo._PhotoImage__photo.zoom(4)
w = Label(root, image=photo)
w.pack()

coordinates = []

root.bind("<Right>", ml.next_photo)
root.bind("<Left>", ml.previous_photo)
w.bind("<Button-1>", ml.callback)

root.mainloop()

## Automatic Point Labeling

In [ ]:
from src.external.labeling import auto_labeling as al

### Single Shot

In [ ]:
file_idx = 0
tv = GetTV('tv_images/l-mode')
files = tv.list_files()
[inverted,radii,elevation,frames,times,vid_frames,vid_times,vid] = tv.load_all(files[file_idx])

In [ ]:
# initialize values
x_0 = np.array([1.30,-1.05]) # x-point location
l_0 = np.array([1.05,-1.21]) # left radiation point location
r_0 = np.array([1.48,-1.25]) # right radiation point location

centers_0 = np.array([al.convert_center(radii,elevation,x_0),
                      al.convert_center(radii,elevation,l_0),
                      al.convert_center(radii,elevation,r_0)])

radius = 6 # for some reason, 6 just works the best while every other value doesn't work
threshold = 0.05 # intensity value threshold for points to update their position
merge_threshold = 20 # merge threshold for radiation point and X-point, should be on factor of 2X radius
distance_threshold = 10 # distance from emission line for corner to be considered a radiation point

### Iterate Across All Shots

In [ ]:
file_idx = 0
tv = GetTV('tv_images/l-mode')
files = tv.list_files()
[inverted,radii,elevation,frames,times,vid_frames,vid_times,vid] = tv.load_all(files[file_idx])

In [ ]:
# initialize values
x_0 = np.array([1.30,-1.05]) # x-point location
l_0 = np.array([1.05,-1.21]) # left radiation point location
r_0 = np.array([1.48,-1.25]) # right radiation point location

centers_0 = np.array([al.convert_center(radii,elevation,x_0),
                      al.convert_center(radii,elevation,l_0),
                      al.convert_center(radii,elevation,r_0)])

radius = 6 # for some reason, 6 just works the best while every other value doesn't work
threshold = 0.05 # intensity value threshold for points to update their position
merge_threshold = 20 # merge threshold for radiation point and X-point, should be on factor of 2X radius
distance_threshold = 10 # distance from emission line for corner to be considered a radiation point

## Semi-Supervised Point Labeling

In [ ]:
file_path = Path('../out/hdf5')
model_file_path = Path('../models')
file_name = 'inversion_manual.hdf5'
file_name = file_path / file_name

tv = GetTV()
files = tv.list_files()
elevation = tv.load(files[0], 'elevation')[0]
radii = tv.load(files[0], 'radii')[0]

In [ ]:
with h5py.File(file_name, 'r') as f:
    inverted = f['inverted'][:]
    points = f['points'][:]

In [ ]:
image = inverted.copy()
X_train, X_test_raw, y_train, y_test = train_test_split(image, points, test_size=0.2)
X_train = X_train.reshape((len(X_train), -1))
X_test = X_test_raw.reshape((len(X_test_raw), -1))

In [ ]:
print(X_test.shape)
print(X_train.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
regr = LinearRegression() # try ridge regression
regr.fit(X_train, y_train)
r_sq = regr.score(X_test, y_test)
r_predict = regr.predict(X_test)
print(r_sq)

In [ ]:
print(f"RMS (cm) : {np.sqrt(mean_squared_error(y_test, r_predict))*100}")

In [ ]:
image2 = image.reshape((len(image), -1))
print(image2.shape)

In [ ]:
regr_dump = LinearRegression()
regr_dump.fit(image2, points)

In [ ]:
model_filename = model_file_path / Path('lr_supervised_mdl').with_suffix('.pkl')
pickle.dump(regr_dump, open(model_filename, 'wb'))

In [ ]:
model_filename = model_file_path / Path('lr_supervised_mdl').with_suffix('.pkl')
loaded_model = pickle.load(open(model_filename, 'rb'))